# 导入库|

## 导入pytoch并检查是否为GPU

In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
import numpy as np
import pickle

In [7]:
combined_array = np.load('out_data/combined_array.npy')

total_energy = np.load('out_data/total_energy.npy')

In [17]:
combined_array

array([[1.13779316e+04, 2.72026625e+05, 1.63766479e+02, ...,
        5.05832761e+00, 3.92486575e+00, 2.83953478e+00],
       [1.66335781e+04, 3.85932219e+05, 1.96714127e+02, ...,
        1.42945050e+00, 4.23036947e+00, 2.89753474e+00],
       [1.66431816e+04, 3.86002562e+05, 1.96775146e+02, ...,
        4.74763596e+00, 3.93023353e+00, 2.84459229e+00],
       ...,
       [1.12181348e+04, 2.60452359e+05, 1.60908371e+02, ...,
        1.36060946e+01, 3.45040872e+00, 2.76416725e+00],
       [1.12181338e+04, 2.60452359e+05, 1.60908356e+02, ...,
        1.36070126e+01, 3.45040643e+00, 2.76417683e+00],
       [1.24861270e+04, 2.94673500e+05, 1.69455948e+02, ...,
        2.06497536e+01, 3.18603619e+00, 2.72785907e+00]])

In [10]:
with open('out_data/total_elec.pkl', 'rb') as file:
    total_elec = pickle.load(file)
with open('out_data/total_spms.pkl', 'rb') as file:
    total_spms = pickle.load(file)
with open('out_data/atom_properties.pkl', 'rb') as file:
    atom_properties = pickle.load(file)
with open('out_data/total_out_basics.pkl', 'rb') as file:
    total_out_basics = pickle.load(file)

## 2d 3d数据

In [19]:
elec = []
spms = []
atom = []
for i in range(len(total_out_basics)):
    indices = [index - 1 for index in total_out_basics[i][0]]
    
    elec_values = [total_elec[i][index] for index in indices]
    elec.append(elec_values)

    spms_values = [total_spms[i][index] for index in indices]
    spms.append(spms_values)
    
    atom_values = [atom_properties[i][index] for index in indices]
    atom.append(atom_values)
elec = np.array(elec)
spms = np.array(spms)
atom = np.array(atom)

print(elec.shape,spms.shape, atom.shape)

(291, 8, 8, 8, 8) (291, 8, 20, 40) (291, 8, 8)


In [88]:
from torch.utils.data import Dataset, DataLoader ,random_split

class ComplexDataset(Dataset):
    def __init__(self, elec, spms, atom,combined_array, energy):
        self.elec = torch.tensor(elec, dtype=torch.float32)  # 将elec转换为张量
        self.spms = torch.tensor(spms, dtype=torch.float32)  # 将spms转换为张量
        self.atom = torch.tensor(atom, dtype=torch.float32)  # atom转换为张量
        self.molecular = torch.tensor(combined_array, dtype=torch.float32)  # atom转换为张量
        self.energy = torch.tensor(energy, dtype=torch.float32)  # 将energy转换为张量

    def __len__(self):
        return len(self.energy)  # 返回样本总数

    def __getitem__(self, idx):
        # 根据索引返回单个样本
        return self.elec[idx], self.spms[idx], self.atom[idx], self.molecular[idx], self.energy[idx]

# 创建ComplexDataset实例
# dataset = ComplexDataset(elec, spms, atom,combined_array, total_energy)


In [71]:
with open('test_dataset/out_data/total_elec.pkl', 'rb') as file:
    total_elec2 = pickle.load(file)
with open('test_dataset/out_data/total_spms.pkl', 'rb') as file:
    total_spms2 = pickle.load(file)
with open('test_dataset/out_data/atom_properties.pkl', 'rb') as file:
    atom_properties2 = pickle.load(file)
with open('test_dataset/out_data/total_out_basics.pkl', 'rb') as file:
    total_out_basics2 = pickle.load(file)
combined_array2 = np.load('test_dataset/out_data/combined_array.npy')

total_energy2 = np.load('test_dataset/out_data/total_energy.npy')


elec2 = []
spms2 = []
atom2 = []
for i in range(len(total_out_basics2)):
    indices = [index - 1 for index in total_out_basics2[i][0]]
    
    elec_values = [total_elec2[i][index] for index in indices]
    elec2.append(elec_values)

    spms_values = [total_spms2[i][index] for index in indices]
    spms2.append(spms_values)
    
    atom_values = [atom_properties2[i][index] for index in indices]
    atom2.append(atom_values)
elec2 = np.array(elec2)
spms2 = np.array(spms2)
atom2 = np.array(atom2)

print(elec2.shape,spms2.shape, atom2.shape)

## 定义数据集

In [101]:
dataset = ComplexDataset(elec, spms, atom,combined_array, total_energy)

test_dataset = ComplexDataset(elec2, spms2, atom2,combined_array2, total_energy2)
# 在创建DataLoader时使用
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(dataset, batch_size=8, shuffle=False)


In [86]:
from torch import nn
import torch.nn.functional as F

class SimplifiedConvNet(nn.Module):
    def __init__(self):
        super(SimplifiedConvNet, self).__init__()
        # 2D数据的卷积层
        self.conv2d_1 = nn.Conv2d(8, 32, kernel_size=3, stride=1, padding=1)
        self.bn2d_1 = nn.BatchNorm2d(32)
        self.maxpool2d_1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2d_2 = nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=1)
        self.bn2d_2 = nn.BatchNorm2d(64)
        self.maxpool2d_2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 3D数据的卷积层
        self.conv3d_1 = nn.Conv3d(8, 32, kernel_size=2, stride=1, padding=1)
        self.bn3d_1 = nn.BatchNorm3d(32)
        self.maxpool3d_1 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.conv3d_2 = nn.Conv3d(32, 64, kernel_size=2, stride=1, padding=1)
        self.bn3d_2 = nn.BatchNorm3d(64)
        self.maxpool3d_2 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.fc_1d1 = nn.Linear(8 * 8, 128)
        self.fc_1d2 = nn.Linear(128, 256)
        self.bn1d_1 = nn.BatchNorm1d(256)
        
        self.bn0 = nn.BatchNorm1d(12)
        
        
        self.dropout2d = nn.Dropout(p=0.4)
        self.dropout3d = nn.Dropout(p=0.5)
        self.dropout1d = nn.Dropout(p=0.1)
        self.dropout0d = nn.Dropout(p=0.05)

        self.dropout = nn.Dropout(p=0.3)

        self.weight_2d = nn.Parameter(torch.ones(1))
        self.weight_3d = nn.Parameter(torch.ones(1))
        self.weight_1d = nn.Parameter(torch.ones(1))
        self.weight_0d = nn.Parameter(torch.ones(1))


        self.fc1 = nn.Linear(512 + 3200 + 256 +12, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 1)


    def forward(self, x3d, x2d, x1d ,x0d):
        
        # 处理3D数据
        x3d = F.relu(self.bn3d_1(self.conv3d_1(x3d)))
        x3d = self.maxpool3d_1(x3d)
        x3d = F.relu(self.bn3d_2(self.conv3d_2(x3d)))
        x3d = self.maxpool3d_2(x3d)
        x3d = x3d.flatten(1)
        
        # 处理2D数据
        x2d = F.relu(self.bn2d_1(self.conv2d_1(x2d)))
        x2d = self.maxpool2d_1(x2d)
        x2d = F.relu(self.bn2d_2(self.conv2d_2(x2d)))
        x2d = self.maxpool2d_2(x2d)
        x2d = x2d.flatten(1)

        # 处理1D数据
        x1d = x1d.view(x1d.size(0), -1)
        x1d = F.relu(self.fc_1d1(x1d))
        x1d = F.relu(self.fc_1d2(x1d))
        x1d = self.bn1d_1(x1d)

        x0d = self.bn0(x0d)

        x3d = self.dropout3d(x3d)
        x2d = self.dropout2d(x2d)
        x1d = self.dropout1d(x1d)
        x0d = self.dropout0d(x0d)


        weighted_3d = self.weight_2d * x3d
        weighted_2d = self.weight_3d * x2d
        weighted_1d = self.weight_1d * x1d
        weighted_0d = self.weight_0d * x0d
        
        combined = torch.cat((weighted_3d, weighted_2d, weighted_1d, weighted_0d), dim=1)

        x = F.relu(self.fc1(combined))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = F.relu(self.fc5(x))
        x = self.fc6(x)

        return x


In [102]:
%%time
import torch
import torch.optim as optim
from torch.optim import lr_scheduler

# 用您的模型和数据初始化函数
model = SimplifiedConvNet().to(device)

epochs=1000
learning_rate=0.01
# 定义损失函数
criterion = nn.MSELoss()

# 定义优化器和学习率衰减
l2_lambda = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)
scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)  # 每100个epoch学习率乘以0.1

for epoch in range(epochs):
    if epoch < 500:lambda_weight=0.000001
    else :lambda_weight=0.00001
    # 训练阶段
    model.train()
    running_train_loss = 0.0
    for inputs_3d, inputs_2d, inputs_1d, inputs_0d, targets in train_dataloader:
        inputs_3d, inputs_2d, inputs_1d, inputs_0d, targets = inputs_3d.to(device), inputs_2d.to(device), inputs_1d.to(device), inputs_0d.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs_3d, inputs_2d, inputs_1d, inputs_0d).squeeze(1)
        loss = criterion(outputs, targets)
        weight_reg = lambda_weight * (model.weight_2d.norm(2) + model.weight_3d.norm(2) + model.weight_1d.norm(2))
        loss += weight_reg

        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train_dataloader)

    # 测试阶段
    model.eval()
    running_test_loss = 0.0
    with torch.no_grad():
        for inputs_3d, inputs_2d, inputs_1d, inputs_0d, targets in test_dataloader:
            inputs_3d, inputs_2d, inputs_1d, inputs_0d, targets = inputs_3d.to(device), inputs_2d.to(device), inputs_1d.to(device), inputs_0d.to(device), targets.to(device)
            outputs = model(inputs_3d, inputs_2d, inputs_1d, inputs_0d).squeeze(1)
            loss = criterion(outputs, targets)
            running_test_loss += loss.item()

    avg_test_loss = running_test_loss / len(test_dataloader)

    print(f'Epoch [{epoch + 1}/{epochs}], Training Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}')
    
    # 更新学习率
    scheduler.step()

print('Finished Training and Evaluation')


Epoch [1/1000], Training Loss: 683.5001, Test Loss: 71.8016
Epoch [2/1000], Training Loss: 76.1317, Test Loss: 47.7777
Epoch [3/1000], Training Loss: 55.7123, Test Loss: 18.5472
Epoch [4/1000], Training Loss: 46.6323, Test Loss: 27.7298
Epoch [5/1000], Training Loss: 60.3479, Test Loss: 101.8864
Epoch [6/1000], Training Loss: 55.3021, Test Loss: 70.6981
Epoch [7/1000], Training Loss: 47.5306, Test Loss: 433.8249
Epoch [8/1000], Training Loss: 67.2500, Test Loss: 19.1521
Epoch [9/1000], Training Loss: 112.6141, Test Loss: 116.6367
Epoch [10/1000], Training Loss: 55.7968, Test Loss: 126.9799
Epoch [11/1000], Training Loss: 48.8314, Test Loss: 9.8331
Epoch [12/1000], Training Loss: 44.5098, Test Loss: 8.7931
Epoch [13/1000], Training Loss: 33.7198, Test Loss: 28.3560
Epoch [14/1000], Training Loss: 57.5289, Test Loss: 27.0064
Epoch [15/1000], Training Loss: 62.2241, Test Loss: 86.7103
Epoch [16/1000], Training Loss: 82.7556, Test Loss: 17.5777
Epoch [17/1000], Training Loss: 43.7670, Test

KeyboardInterrupt: 

In [107]:
model.eval()
for inputs_3d, inputs_2d, inputs_1d, inputs_0d, targets in train_dataloader:
    inputs_3d, inputs_2d, inputs_1d, inputs_0d, targets = inputs_3d.to(device), inputs_2d.to(device), inputs_1d.to(device), inputs_0d.to(device), targets.to(device)
    
    optimizer.zero_grad()
    outputs = model(inputs_3d, inputs_2d, inputs_1d, inputs_0d).squeeze(1)
    loss = criterion(outputs, targets)
    weight_reg = lambda_weight * (model.weight_2d.norm(2) + model.weight_3d.norm(2) + model.weight_1d.norm(2))
    loss += weight_reg
    running_train_loss += loss.item()

avg_train_loss = running_train_loss / len(train_dataloader)
        
running_test_loss/ len(train_dataloader)

70.15827178955078

In [104]:
outputs

tensor([27.0433, 15.2809, 28.0996], device='cuda:0')

In [105]:
targets

tensor([34.9976, 18.8765, 38.3754], device='cuda:0')